## 1. Introduction

## 2. Setup and Imports

## 3. Data Loading

## 4. Data Preprocessing: Generating Visibility Levels

## 5. Exploratory Data Analysis

## 6. Classification Model Training (Cross-Validation)

## 7. Initial Conformal Prediction (Single-Level Baseline)

## 8. Visibility-Aware Conformal Prediction

## 9. Visibility Estimator Training

## 10. Integrated Testing: Visbility Estimator + CP

## 11. Predicting Visibility CP Resolution

## 12. Visualization and Interpretation

## 13. Discussion and Conclusions

## 14. References